In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("AmesHousing.txt",sep="\t")

In [6]:

print(df.shape)
df.head()

(2930, 82)


,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


#### Features Transformation

- Remove columns with Null values
- Convert String columns to Categorical
- Convert Categorical columns to Dummy columns
- Convert columns have Year or date to No_of_years_passed or No_of_days etc. which makes more sense to the model


**Handling Null values in row and columns**

Two approaches
- Remove the entire row
- Replace the null value with a descriptive statistics from the column like mean, max, min etc.

In [17]:
# Finds all the columns with no NaN values
null_cols = df.isna().sum()
not_null_index = null_cols[null_cols ==0].index

In [20]:
# Creates a new df with all non-nan columns
nndf = df[not_null_index].copy()

In [21]:
nndf.shape

(2930, 55)

In [24]:
df_size = int(len(nndf)/2)
print(df_size)
train = nndf.iloc[:df_size]
print(train.shape)
test = nndf.iloc[df_size+1:]
print(test.shape)

1465
(1465, 55)
(1464, 55)


In [40]:
object_cols = nndf.dtypes[nndf.dtypes == "object"].index

In [27]:
train['Utilities'].value_counts()

AllPub    1462
NoSewr       2
NoSeWa       1
Name: Utilities, dtype: int64

In [28]:
# Convert string col to category
train['Utilities'] = train['Utilities'].astype('category')

c:\pyvirtualenv\ml\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
# Check the column is converted to Category - with 3 categories
train['Utilities']

[AllPub, NoSewr, NoSeWa]
Categories (3, object): [AllPub, NoSewr, NoSeWa]

In [37]:
# Utilities is converted to Category
train['Utilities'].dtypes

CategoricalDtype(categories=['AllPub', 'NoSeWa', 'NoSewr'], ordered=False)

In [42]:
for col in object_cols:
    print(f"Column----------> {col} \n\n")
    print(train[col].value_counts())

Column----------> MS Zoning 


RL         1128
RM          232
FV           83
RH           11
C (all)      10
I (all)       1
Name: MS Zoning, dtype: int64
Column----------> Street 


Pave    1460
Grvl       5
Name: Street, dtype: int64
Column----------> Lot Shape 


Reg    942
IR1    480
IR2     36
IR3      7
Name: Lot Shape, dtype: int64
Column----------> Land Contour 


Lvl    1340
Bnk      54
HLS      45
Low      26
Name: Land Contour, dtype: int64
Column----------> Utilities 


AllPub    1462
NoSewr       2
NoSeWa       1
Name: Utilities, dtype: int64
Column----------> Lot Config 


Inside     1061
Corner      259
CulDSac      90
FR2          48
FR3           7
Name: Lot Config, dtype: int64
Column----------> Land Slope 


Gtl    1402
Mod      55
Sev       8
Name: Land Slope, dtype: int64
Column----------> Neighborhood 


NAmes      239
OldTown    140
CollgCr    122
Somerst    102
NridgHt     91
SawyerW     84
Gilbert     82
Edwards     80
NWAmes      76
Sawyer      76
BrkSide   

In [43]:
# Converts all object data type to category dtype
for col in object_cols:
    train[col] = train[col].astype('category')

c:\pyvirtualenv\ml\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [47]:
train.dtypes.unique()

array([dtype('int64'),
       CategoricalDtype(categories=['C (all)', 'FV', 'I (all)', 'RH', 'RL', 'RM'], ordered=False),
       CategoricalDtype(categories=['Grvl', 'Pave'], ordered=False),
       CategoricalDtype(categories=['IR1', 'IR2', 'IR3', 'Reg'], ordered=False),
       CategoricalDtype(categories=['Bnk', 'HLS', 'Low', 'Lvl'], ordered=False),
       CategoricalDtype(categories=['AllPub', 'NoSeWa', 'NoSewr'], ordered=False),
       CategoricalDtype(categories=['Corner', 'CulDSac', 'FR2', 'FR3', 'Inside'], ordered=False),
       CategoricalDtype(categories=['Gtl', 'Mod', 'Sev'], ordered=False),
       CategoricalDtype(categories=['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr',
                  'CollgCr', 'Crawfor', 'Edwards', 'Gilbert', 'Greens',
                  'IDOTRR', 'MeadowV', 'Mitchel', 'NAmes', 'NPkVill', 'NWAmes',
                  'NoRidge', 'NridgHt', 'OldTown', 'SWISU', 'Sawyer',
                  'SawyerW', 'Somerst', 'StoneBr', 'Timber', 'Veenker'],
       

#### Disadvantages of Categorical column in Linear Regression

When we convert a column to the categorical data type, pandas assigns a number from 0 to n-1 (where n is the number of unique values in a column) for each value. The drawback with this approach is that one of the assumptions of linear regression is violated here. Linear regression operates under the assumption that the features are linearly correlated with the target column. For a categorical feature, however, there's no actual numerical meaning to the categorical codes that pandas assigned for that column. An increase in the Utilities column from 1 to 2 has no correlation value with the target column, and the categorical codes are instead used for uniqueness and exclusivity (the category associated with 0 is different than the one associated with 1).

The common solution is to use a technique called dummy coding. Instead of having a single column with n integer codes, we have n binary columns. Here's what that would look like for the Utilities column:



    
    
<table>
    <tr><td>Utilities_AllPub</td><td>    Utilities_NoSewr</td><td>	Utilities_NoSeWa</td></tr>
    <tr><td>1	</td><td>0	</td><td>0</td></tr>
    <tr><td>1	</td><td>0	</td><td>0</td></tr>
    <tr><td>1	</td><td>0	</td><td>0</td></tr>
    <tr><td>1	</td><td>0	</td><td>0</td></tr>
</table>

In [55]:
# Split Utilities columns into as many columns as its unique values
# could also use pd.get_dummies()
utility_codes = train['Utilities'].unique()
for code in utility_codes:
    train["Utilities_"+code] = train["Utilities"].apply(lambda val: 1 if val == code else 0)

c:\pyvirtualenv\ml\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [62]:
cols = ["Utilities_"+code for code in utility_codes]
cols.append("Utilities")
train[cols]

,Utilities_AllPub,Utilities_NoSewr,Utilities_NoSeWa,Utilities
945,0,1,0,NoSewr
1013,0,1,0,NoSewr


In [ ]:
#train['Paved Drive']
pd.get_dummies(train['Paved Drive'], prefix="Paved_Drive")

In [68]:
train.columns

Index(['Order', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Area', 'Street',
       'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config', 'Land Slope',
       'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Overall Qual', 'Overall Cond', 'Year Built',
       'Year Remod/Add', 'Roof Style', 'Roof Matl', 'Exterior 1st',
       'Exterior 2nd', 'Exter Qual', 'Exter Cond', 'Foundation', 'Heating',
       'Heating QC', 'Central Air', '1st Flr SF', '2nd Flr SF',
       'Low Qual Fin SF', 'Gr Liv Area', 'Full Bath', 'Half Bath',
       'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd',
       'Functional', 'Fireplaces', 'Paved Drive', 'Wood Deck SF',
       'Open Porch SF', 'Enclosed Porch', '3Ssn Porch', 'Screen Porch',
       'Pool Area', 'Misc Val', 'Mo Sold', 'Yr Sold', 'Sale Type',
       'Sale Condition', 'SalePrice', 'Utilities_AllPub', 'Utilities_NoSewr',
       'Utilities_NoSeWa'],
      dtype='object')

In [70]:
train.shape

(1465, 58)

In [71]:
train.isnull().sum()

Order               0
PID                 0
MS SubClass         0
MS Zoning           0
Lot Area            0
Street              0
Lot Shape           0
Land Contour        0
Utilities           0
Lot Config          0
Land Slope          0
Neighborhood        0
Condition 1         0
Condition 2         0
Bldg Type           0
House Style         0
Overall Qual        0
Overall Cond        0
Year Built          0
Year Remod/Add      0
Roof Style          0
Roof Matl           0
Exterior 1st        0
Exterior 2nd        0
Exter Qual          0
Exter Cond          0
Foundation          0
Heating             0
Heating QC          0
Central Air         0
1st Flr SF          0
2nd Flr SF          0
Low Qual Fin SF     0
Gr Liv Area         0
Full Bath           0
Half Bath           0
Bedroom AbvGr       0
Kitchen AbvGr       0
Kitchen Qual        0
TotRms AbvGrd       0
Functional          0
Fireplaces          0
Paved Drive         0
Wood Deck SF        0
Open Porch SF       0
Enclosed P